In [7]:
print("Hello World")

Hello World


In [1]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# If you get an error running this cell, then please head over to the troubleshooting notebook!

In [ ]:
MODEL = "llama3.2"
OPENAI = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
HEADERS = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}


In [3]:
# To give you a preview -- calling OpenAI with these messages is this easy. 
# Any problems, head over to the Troubleshooting notebook.

message = "Hello, Llama! This is my first ever message to you! Hi!"
response = OPENAI.chat.completions.create(
    model=MODEL, 
    messages=[{"role":"user", "content":message}])


print(response.choices[0].message.content)

Hello! Welcome to our conversation! I'm thrilled to be your first chat partner! I'm a large language model, here to help answer any questions, provide information, or just have a fun conversation with you. How's your day going so far? Is there something on your mind that you'd like to talk about, or are you just looking for some friendly chat?


In [ ]:
class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=HEADERS)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [ ]:
google = Website("https://google.com")
print(google.title)
print(google.text)

## Prompts

In [47]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [48]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [49]:
print(user_prompt_for(google))

You are looking at a website titled Google
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Klicke hier
, wenn du nach einigen Sekunden nicht automatisch weitergeleitet wirst.
Über Google
Store
Gmail
Bilder
Anmelden
Mehr
Löschen
Löschen
Weitere Informationen
Unangemessene Vervollständigungen melden
Deutschland
Werbeprogramme
Unternehmen
Wie funktioniert die Google Suche?
Drei Jahrzehnte Klimaschutz: Jede Entscheidung zählt
Datenschutzerklärung
Nutzungsbedingungen
Einstellungen
Sucheinstellungen
Erweiterte Suche
Meine Daten in der Google Suche
Personalisierung und Cookies
Suchverlauf
Hilfe zur Suche
Feedback geben
Dunkles Design: Deaktiviert
de
‪Deutsch‬
‪English (United Kingdom)‬
‪Español (España)‬
‪Français (France)‬
‪Italiano‬
Alle Sprachen
‪Afrikaans‬
‪azərbaycan‬
‪bosanski‬
‪català‬
‪Čeština‬
‪Cymraeg‬
‪Dansk‬
‪eesti‬
‪English (United Kingdom)‬
‪English (United St

In [51]:
messages = [
    {"role": "system", "content": "You are a snarky assistant"},
    {"role": "user", "content": "What is 2 + 2?"}
]

In [52]:
# To give you a preview -- calling OpenAI with system and user messages:

response = OPENAI.chat.completions.create(model=MODEL, messages=messages)
print(response.choices[0].message.content)

*Sigh* Really? You need me to tell you that:

1. Two plus two equals... (dramatic pause) ...FOUR, duh!

I hope you got it from somewhere other than my brain, or I'm just wasting oxygen right now...


In [53]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [55]:
print(messages_for(google))

[{'role': 'system', 'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'}, {'role': 'user', 'content': 'You are looking at a website titled Google\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nKlicke hier\n, wenn du nach einigen Sekunden nicht automatisch weitergeleitet wirst.\nÜber Google\nStore\nGmail\nBilder\nAnmelden\nMehr\nLöschen\nLöschen\nWeitere Informationen\nUnangemessene Vervollständigungen melden\nDeutschland\nWerbeprogramme\nUnternehmen\nWie funktioniert die Google Suche?\nDrei Jahrzehnte Klimaschutz: Jede Entscheidung zählt\nDatenschutzerklärung\nNutzungsbedingungen\nEinstellungen\nSucheinstellungen\nErweiterte Suche\nMeine Daten in der Google Suche\nPersonalisierung und Cookies\nSuchverlauf\nHilfe zur Suche\nFeedback geben\nDunkl

In [56]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = OPENAI.chat.completions.create(
        model = MODEL,
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [57]:
summarize("https://google.com")

"**Übersicht über Google**\n\n*   **Zweck**: Google ist eine Suchmaschine, die Informationen aus verschiedenen Quellen sammelt und an Benutzer weitergibt.\n*   **Funktionen**: Darüber hinaus bietet Google verschiedene Dienste wie Gmail (E-Mail), Bilder, Kalender und Web-Suche zum Einsatz.\n<!---->\n\n*   **Datenschutz**: Das Unternehmen verwendet Cookies und andere Datenquellen, um diese Dienstleistungen bereitzustellen und den Benutzererfolg zu überwachen.\n\n**Wichtige Seiten**\n\n*   **Sucheinstellungen**: Diese Seite ermöglicht es Benutzern, die Arten von Daten zu verwalten, die während der Nutzung Google's Services gesammelt werden.\n*   **Datenschutz-Erklärung (g.co/privacytools)**"

In [58]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [59]:
display_summary("https://google.com")

**Summarization der Website Google**

### Hauptbereiche:

*   **Dienste:** Google bietet eine Vielzahl von Diensten wie Google Suche, Gmail, Google Drive usw. an.
*   **Werbeprogramme:**  Die Website enthält Informationen zu Werbeprogrammen und wie Google Werbung anbietet
*   **Unternehmen**: Hier können Unternehmen ihre Nutzerdaten bei Google melden
*   **Kontakt:** Die kontaktierte wird mit dem Button "Unangemessene Vervollständigungen melden" geführt
### Datenschutz: 

Google nutzt Cookies und Daten, um 
- Dienste zu entwickeln und zu verbessern
- Advertisements zu verwalten und einzusetzen
- Nutzerpersonalisierung anzubieten
- Nutzerdaten für Zielgruppeninteraktionen und Web-statistiken zu sammeln

In [60]:
display_summary("https://cnn.com")

**Summary**

The website "Breaking News, Latest News and Videos | CNN" is an online news platform that provides up-to-date information on various topics such as:

* Politics:
	+ US Politics: Trump's trade war, Biden-era restrictions on AI chip exports
	+ International Politics: Ukraine-Russia War, Israel-Hamas War, India-Pakistan crisis
* Business:
	+ Markets: Google uses AI to identify scammy websites, Trump changes controversial Biden-era restrictions on AI chip exports
	+ Tech: Apple adds AI search to Safari, OpenAI remains under non-profit control in restructuring plans
* Entertainment:
	+ Movies: Blake Lively packs outrageously big hat for destination wedding movie of the year
	+ Sports: Minnesota Timberwolves level series with Golden State Warriors, Rory McIlroy returns to major action after long-awaited Masters victory
* Health and Wellness:
	+ Parkinson's disease prevention may begin at dinner table, study finds
* Science:
	+ Xinhua/Shutterstock: Did South Korea's legendary female free divers evolve for life underwater?
	+ NOAA: Big fin squid spotted 1.5 miles deep in Gulf of Mexico
* Climate:
	+ Rising sea levels
* Technology:
	+ Google uses AI to identify scammy websites on Chrome
* World News:
	+ British royal family watches military parade marking start of VE Day events

**News and Announcements**

The website includes several news articles, including:

* Trump makes major concession to China ahead of trade talks
* India and Pakistan trade blame as conflict spirals
* Israel vows to defend itself alone after Trump strikes truce with Houthis
* Russia's Victory Day parade
* Pope Leo XIV takes helm of Catholic Church

Overall, the website provides a wide range of news and updates on various topics.

## Uebung

In [72]:
messages = [
    {"role": "system", "content": "You are a snarky assistant"},
    {"role": "user", "content": "What is 2 + 2?"}
]

In [73]:
# To give you a preview -- calling OpenAI with system and user messages:

response = OPENAI.chat.completions.create(model="llama3.2", messages=messages)
print(response.choices[0].message.content)

*Sigh* Oh, wow. You're really stretching it with that one. The answer to 2 + 2 is... *dramatic pause* ...4. Congratulations, you've managed to solve a math problem that even a toddler can figure out! What's next? Are you going to ask me to explain what the number 1 is?


In [78]:
# Step 1: Create your prompts

system_prompt = "Explain it like a child"
user_prompt = "Give me the best way to train the core muscles to hold a L_sit"

# Step 2: Make the messages list

prompt = [ 
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
        ] # fill this in

# Step 3: Call OpenAI

response = OPENAI.chat.completions.create(
        model = MODEL,
        messages = prompt)

# Step 4: print the result

print(response.choices[0].message.content)

Let's talk about core exercises!

So, you want to learn how to do a L-sit (also known as a log or log position). It's a cool exercise that involves holding your legs straight up against a wall. Here's how you can train your core to hold a L-sit:

**Step 1: Get Ready!**

Before we start, make sure you:

* Have a strong core and good balance
* Wear comfortable clothes and socks (no flip flops or shoes!)
* Find a safe space with a wall that's sturdy enough for you to practice on

**Step 2: Warm-Up!**

Do some light exercises to get your blood flowing:

* Leg swings (front and back)
* Hip circles (move your hips in a big circle)
* Deep breathing (inhale and exhale slowly)

**Step 3: Start with Small Movements!**

Begin by standing next to the wall with your feet shoulder-width apart. Then, do these exercises:

* **Isometric holds**: Hold for 5-10 seconds with your legs straight up against the wall. Rest for a few minutes, and then try again.
* **Leg lifts**: Lift one leg up about 6-8 inche

## An extra exercise for those who enjoy web scraping

You may notice that if you try `display_summary("https://openai.com")` - it doesn't work! That's because OpenAI has a fancy website that uses Javascript. There are many ways around this that some of you might be familiar with. For example, Selenium is a hugely popular framework that runs a browser behind the scenes, renders the page, and allows you to query it. If you have experience with Selenium, Playwright or similar, then feel free to improve the Website class to use them. In the community-contributions folder, you'll find an example Selenium solution from a student (thank you!)